In [47]:
import pandas as pd

In [48]:
full=pd.read_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\full_data.parquet')

In [49]:
full['Ratting']=full['recommend']+full['sentiment_analysis']

In [50]:
prueba=full[['user_id','title','Ratting']]

In [51]:
muestra=prueba.sample(frac=0.04, random_state=42)

In [52]:
from surprise.model_selection import GridSearchCV, train_test_split
from surprise import SVD, Dataset, Reader, accuracy


In [53]:
reader=Reader(rating_scale=(0, 3)) 
data=Dataset.load_from_df(muestra,reader)

In [55]:
param_grid = {
    'n_factors': [0, 100],
    'n_epochs': [0, 50],
    'lr_all': [0, 0.01],
    'reg_all': [0, 0.05],
    'random_state': [0, 42]
}
model = SVD()
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)
best_params = grid_search.best_params['rmse']
print("Mejores parámetros encontrados:", best_params)

Mejores parámetros encontrados: {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0, 'random_state': 42}


In [54]:
from surprise import accuracy

trainset, testset=train_test_split(data, test_size=0.2)
model=SVD(n_factors=100,n_epochs=50,lr_all=0.01,reg_all=0,random_state=42)
model.fit(trainset)
predictions=model.test(testset)

rmse=accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.5529
RMSE: 0.5528687229241392


In [56]:
def recomendaciones_usuario(usuario: str):
    predicciones=[]
    if usuario not in muestra['user_id'].unique():
        juegos_aleatorios = list(muestra['title'].sample(5))
        return {f'El usuario {usuario} no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria: {juegos_aleatorios}.'}
    else:
        diccionario = {}
        juegos_valorados = muestra[muestra['user_id'] == usuario]['title'].unique()
        todos_los_juegos = muestra['title'].unique()
        juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))
        for juego in juegos_no_valorados:
            prediccion=model.predict(usuario, juego)
            predicciones.append(prediccion)
        recomendaciones=sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 
        cont = 1
        for recomendacion in recomendaciones:
            diccionario[f'Top {cont}'] = recomendacion.iid
            cont += 1
            
        return {f'Usuario: {usuario}': diccionario}

In [57]:
user=prueba['user_id'].unique()
diccionario = {}
for x in user[:5]:
    diccionario[x] = recomendaciones_usuario(x)
diccionario

{'Gaaaaary': {'Usuario: Gaaaaary': {'Top 1': 'Team Fortress 2',
   'Top 2': 'Castle Crashers®',
   'Top 3': 'Max Payne 3',
   'Top 4': 'Crush Crush',
   'Top 5': 'Portal'}},
 'darkedge119': {'Usuario: darkedge119': {'Top 1': 'No More Room in Hell',
   'Top 2': 'Goat Simulator',
   'Top 3': '911: First Responders®',
   'Top 4': 'Echo of Soul',
   'Top 5': 'Revenge of the Titans'}},
 'razarealm': {'Usuario: razarealm': {'Top 1': 'Portal 2',
   'Top 2': 'Counter-Strike: Source',
   'Top 3': "Oddworld: Abe's Oddysee®",
   'Top 4': 'Dungeonland',
   'Top 5': 'Grass Simulator'}},
 'mysterd': {'Usuario: mysterd': {'Top 1': 'Revenge of the Titans',
   'Top 2': 'Brothers - A Tale of Two Sons',
   'Top 3': 'Wolfenstein: The New Order',
   'Top 4': 'Dungeonland',
   'Top 5': 'TinyKeep'}},
 'Pyromanic': {'El usuario Pyromanic no posee ningún item en su biblioteca, por lo que la recomendación será aleatoria: [\'Rust\', \'resident evil 4 / biohazard 4\', \'RaceRoom Racing Experience\', \'Guacamelee!

In [58]:
import pickle
import gzip

In [59]:
muestra.to_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos varios\relacionamiento_muestra.parquet', compression="gzip")

In [60]:
with open(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Modelos\Relaciones_muestra.pkl', 'wb') as file:
    pickle.dump(model, file)